# Topic modeling of job reviews
## Andrew Hall <br><sup>August 28, 2022 <br> Initial script for final project of Metis NLP short course</sup>


In [45]:
import pandas as pd

business_names = ['Adobe', 'Airbnb', 'Amazon', 'Apple', 'Atlassian', 'Bloomberg',
                 'Bytedance', 'Cisco', 'Coinbase', 'Deloitte', 'Goldman-Sachs', 'Google',
                 'IBM', 'Intel', 'Intuit', 'Meta', 'Microsoft', 'Netflix', 'Oracle',
                 'Salesforce', 'SAP-Labs', 'Stripe', 'Twitter', 'Uber', 'Walmart']

#initialize a dictionary
data = {}

#read in each company data with key being name of company
for name in business_names: 
    data[name] = pd.read_csv('data/'+name+'/'+name+'-data.csv')

In [95]:
full_data = []
cols = ['Description', 'Pros', 'Cons', 'Company']
for company in data:
    subset_data = data[company]
    subset_data['Company'] = company
    full_data.append(subset_data[cols])

full_data = pd.concat(full_data)
print("The number of documents is: ", full_data.shape[0])

The number of documents is:  43803


In [96]:
full_data_long = pd.melt(full_data, id_vars = ['Company'], value_vars = ['Description', 'Pros', 'Cons'], var_name = 'Prompt', value_name = 'Output')
print("The number of individual document components is: ", full_data_long.shape[0])

The number of individual document components is:  131409


In [97]:
sum([len(str(d).split(' ')) for d in full_data_long.Output]) > 100000.

True

In [98]:
print("The aggregated number of non-unique terms is: ", sum([len(str(d).split(' ')) for d in full_data_long.Output]))

The aggregated number of non-unique terms is:  1903424


In [76]:
# example call of the data for the first ten entries for Adobe
for d in full_data_long.Output[:10]:
    print(d)

A decent tier 2 company 
Good Company...terrible middle managers
Great place to work
Not a place for work life balance, full of politics.
Work life balance is good
Great benefits and very good wlb
First Impressions 
Gr8 WLB, Management heavy with no direction
Disappointing 
Adobe is amazing. Your managers may not be.


# Create initial vectorized output

In [101]:
from sklearn.feature_selection import CountVectorizer

ImportError: cannot import name 'CountVectorizer' from 'sklearn.feature_selection' (/Users/andrewhall/opt/anaconda3/envs/unsupervised/lib/python3.8/site-packages/sklearn/feature_selection/__init__.py)

______________________
### Unused output

In [130]:
for company in data.keys(): 
    index = list(data.keys()).index(company)
    print('data_'+list(data.keys())[index])

data_Adobe
data_Airbnb
data_Amazon
data_Apple
data_Atlassian
data_Bloomberg
data_Bytedance
data_Cisco
data_Coinbase
data_Deloitte
data_Goldman-Sachs
data_Google
data_IBM
data_Intel
data_Intuit
data_Meta
data_Microsoft
data_Netflix
data_Oracle
data_Salesforce
data_SAP-Labs
data_Stripe
data_Twitter
data_Uber
data_Walmart


In [137]:
for company in data.keys(): 
    index = list(data.keys()).index(company)
    print('data_'+list(data.keys())[index])
    

<class 'pandas.core.frame.DataFrame'>


ValueError: Expected a 1D array, got an array with shape (954, 6)